### 필요한 라이브러리 Import

In [ ]:
import os
import time
import threading
from pymycobot.mycobot280 import MyCobot280
from pymycobot.genre import Angle, Coord

### 로봇 연결

In [ ]:
mc = MyCobot280('/dev/ttyJETCOBOT', 1000000)
mc.thread_lock = True
print("로봇이 연결되었습니다.")

### 로봇의 현재 데이터 읽기 (가장 많이 쓸 코드!)

In [ ]:
# 현재 각도 읽기
angles = mc.get_angles()
print("현재 각도:", angles)
# 현재 좌표 읽기
coords = mc.get_coords()
print("현재 좌표:", coords)
# 인코더 값 읽기
encoders = mc.get_encoders()
print("인코더:", encoders)
# 라디안 값 읽기
radians = mc.get_radians()
print("라디안:", radians)

### 로봇을 초기위치로 이동(Home Pose)

In [ ]:
ANGLE_MIN = [-168, -135, -150, -145, -165, -180, 0]
ANGLE_MAX = [168, 135, 150, 145, 165, 180, 100]

for i in range(7):
    print(f"관절 {i+1}: {ANGLE_MIN[i]} ~ {ANGLE_MAX[i]}도")

### 단일 관절 각도 움직이기 : send_angle(조인트 번호, 목표 각도, 속도)

In [ ]:
# 로봇을 초기 위치로 리셋
initial_angles = [0, 0, 0, 0, 0, 0]
speed = 50

print("로봇을 초기 위치로 리셋합니다.")
mc.send_angles(initial_angles, speed)
mc.set_gripper_value(100, speed) # 그리퍼 열기
time.sleep(3) # 움직임이 완료될 때까지 대기
print("리셋 완료")

### 모든 관절 각도 움직이기 : send_angles(목표 각도 List, 속도)

In [ ]:
# 모든 관절을 지정된 각도로 이동
target_angles = [20, 20, -20, 20, 20, -45]
speed = 50

print(f"모든 관절을 {target_angles}로 이동합니다.")
mc.send_angles(target_angles, speed)
time.sleep(3) # 움직임이 완료될 때까지 대기
# 초기 위치로 복귀
print("초기 위치로 복귀합니다.")
mc.send_angles([0, 0, 0, 0, 0, 0], speed)
time.sleep(3) # 움직임이 완료될 때까지 대기

### 좌표로 로봇 제어하기 : send_coords(목표 좌표 List, 속도, 모드) cf. coordinate=좌표

In [ ]:
# 현재 좌표 확인
current_coords = mc.get_coords()
print("현재 좌표:", current_coords)

# 1. 먼저 Z축을 낮추기
work_coords = current_coords.copy()
work_coords[2] -= 50 # Z를 50mm 내리기
print(f"Z축을 {work_coords[2]}로 내립니다.")
mc.send_coords(work_coords, 30, 0)
time.sleep(2)

# 2. X 좌표 이동
x_coords = work_coords.copy()
x_coords[0] += 20 # X + 20mm
print(f"X 좌표를 {x_coords[0]}로 이동합니다.")
mc.send_coords(x_coords, 30, 0)
time.sleep(2)

# 3. Y 좌표 이동
y_coords = x_coords.copy()
y_coords[1] -= 20 # Y - 20mm
print(f"Y 좌표를 {y_coords[1]}로 이동합니다.")
mc.send_coords(y_coords, 30, 0)
time.sleep(2)

# 4. 최종 좌표 확인
final_coords = mc.get_coords()
print("최종 좌표:", final_coords)

# 5. 초기 위치로 복귀
print("초기 위치로 복귀합니다.")
mc.send_angles([0, 0, 0, 0, 0, 0], 50)
time.sleep(3)
print("초기 위치 복귀 완료")

### 모든 좌표로 한번에 이동(Inverse kinematics)

In [ ]:
# 현재 좌표 확인
current_coords = mc.get_coords()
print("현재 좌표:", current_coords)

# 목표 좌표 설정 (현재에서 조금 변경)
target_coords = current_coords.copy()
target_coords[0] += 30 # X + 30mm
target_coords[1] -= 30 # Y - 30mm
target_coords[2] -= 50 # Z - 50mm
print(f"목표 좌표로 이동합니다: {target_coords}")
mc.send_coords(target_coords, 50, 0)
time.sleep(3)

# 초기 좌표로 복귀
print("초기 위치로 복귀합니다.")
mc.send_angles([0, 0, 0, 0, 0, 0], 50)

### 그리퍼 제어

In [ ]:
# 그리퍼 완전히 열기
print("그리퍼를 완전히 엽니다.")
mc.set_gripper_value(100, 50)
time.sleep(1)

# 그리퍼 반쯤 닫기
print("그리퍼를 반쯤 닫습니다.")
mc.set_gripper_value(50, 50)
time.sleep(1)

# 그리퍼 더 닫기
print("그리퍼를 더 닫습니다.")
mc.set_gripper_value(30, 50)
time.sleep(1)

# 그리퍼 완전히 닫기
print("그리퍼를 완전히 닫습니다.")
mc.set_gripper_value(0, 50)
time.sleep(1)

# 그리퍼 다시 열기
print("그리퍼를 다시 엽니다.")
mc.set_gripper_value(100, 50)
time.sleep(1)

### 수동 조작 모드 (주의 : 실행전에 손으로 로봇을 잡고 시작해주세요.)

In [ ]:
# 모터 비활성화
print("전체 모터를 비활성화합니다.")
mc.release_all_servos()
time.sleep(1)

### 자동 조작 모드

In [ ]:
# 모터 활성화
print("전체 모터를 활성화합니다.")
mc.focus_all_servos()
time.sleep(1)

#### https://github.com/elephantrobotics/pymycobot/blob/main/docs/MyCobot_280_en.md